### import Libraries

In [1]:
from langchain_mistralai import MistralAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_mistralai import ChatMistralAI
import os
import json

C:\Users\gadda\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open(r"C:\Users\gadda\OneDrive\Desktop\AI_Resume_Parser\Data\processed\structured_resume.json",'r',encoding='utf-8') as f:
    structured_resume=json.load(f)

In [3]:
structured_resume['name']

'Hayden Smith'

### Resume Text for Embedding

In [4]:
resume_doc_text = f"""
Candidate Name: {structured_resume["name"] }

Skills:
{", ".join(structured_resume["skills"])}

Experience:
{" ".join(structured_resume["experience"])}

Education:
{" ".join(structured_resume["education"])}
"""


### Initialize Embeddings

In [5]:
embeddings=MistralAIEmbeddings(api_key=os.getenv("MISTRAL_API_KEY"),
                   model="mistral-embed")

In [6]:
LLM=ChatMistralAI(model_name='mistral-small',
                 api_key=os.getenv('MISTRAL_API_KEY'),
                 temperature=0.0) 

### create document

In [7]:
resume_doc = Document(
    page_content=resume_doc_text,
    metadata={
        "candidate_name": structured_resume["name"]
       
    }
)


### Create Chromadb Vectorstore

In [8]:
vector_db=Chroma.from_documents(embedding=embeddings,
                  documents=[resume_doc],
                  
                  persist_directory=r"C:\Users\gadda\OneDrive\Desktop\AI_Resume_Parser\Vector_store\Chroma"
)


In [9]:
job_description = """
We are looking for a part-time retail assistant with strong customer service,
communication skills, teamwork abilities, and experience handling cash.
Prior experience in a sports retail environment is preferred.
"""


In [10]:
results = vector_db.similarity_search_with_score(
    job_description,
    k=1
)

matched_doc, similarity_score = results[0]

similarity_score


0.36091989278793335

In [11]:
match_prompt = f"""
You are an AI hiring assistant.

Resume:
{matched_doc.page_content}

Job Description:
{job_description}

Tasks:
1. List matching skills
2. List missing skills
3. Give a suitability score from 0 to 100
4. Provide a short explanation
"""

response = LLM.invoke(match_prompt)
response.content


'### **1. Matching Skills**\n- **Customer service** (experience in canteen and coaching)\n- **Cash handling** (experience in canteen)\n- **Communication** (experience in coaching and umpiring)\n- **Teamwork** (experience in coaching, umpiring, and canteen work)\n- **Organization** (experience in coaching and newspaper delivery)\n- **Problem-solving** (experience in umpiring and coaching)\n\n### **2. Missing Skills**\n- **Sports retail experience** (preferred but not mandatory)\n- **Inventory management** (not explicitly mentioned)\n- **Point-of-sale (POS) system experience** (not mentioned)\n\n### **3. Suitability Score: 75/100**\n### **4. Explanation**\nHayden Smith has strong **customer service, communication, and teamwork** skills, which are essential for a retail assistant role. Their experience in **cash handling** (canteen work) and **problem-solving** (umpiring and coaching) makes them a good fit. However, they lack direct **sports retail experience**, which is preferred but not